# Step 1: Dataset Preparation

The Dataset of Spotify Stock News 

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [62]:
#load the csv file that contains the news from the finhub api 03/2024 to 02/2025
df_news = pd.read_csv("/Users/armandocriscuolo/c2025/data_science_project_2025/code/Data-Science-Project/Data-Question-2-B/news_analysis/spotify_news_2024_2025_finhub.csv")

#print the first 5 rows of the dataframe
print(df_news.head())

print("\n\n-----------------------------------\n\n")

#load the csv file that contains the stock data from the marketstack api 03/2024 to 02/2025
df_stock = pd.read_csv("/Users/armandocriscuolo/c2025/data_science_project_2025/code/Data-Science-Project/Data-Question-2-B/news_analysis/spotify_stock_data_20250323_193740.csv")

#print the first 5 rows of the dataframe
print(df_stock.head())

  category    datetime                                           headline  \
0  company  1740759981  Palantir, Nvidia Nixed But Netflix Remains On ...   
1  company  1740684439  Tracking Chase Coleman's Tiger Global Portfoli...   
2  company  1740664816  Spotify Technology (SPOT) is Attracting Invest...   
3  company  1740658999  YouTube Surpasses Competitors in Streaming and...   
4  company  1740593693  Spotify CEO Wants EU To Penalize Apple For Def...   

          id                                              image related  \
0  132937882  https://media.zenfs.com/en/ibd.com/fc3f416bcfc...    SPOT   
1  132901707  https://static.seekingalpha.com/cdn/s3/uploads...    SPOT   
2  132937883  https://media.zenfs.com/en/zacks.com/bdc2850a4...    SPOT   
3  132937884  https://media.zenfs.com/en/us.finance.gurufocu...    SPOT   
4  132937885  https://media.zenfs.com/en/Benzinga/20dc68a2c9...    SPOT   

         source                                            summary  \
0         Yahoo 

In [63]:
#Lets clean the dataframe to have only the columns we need
print("Spotify Stock Data")
df_stock = df_stock[["date", "open", "high", "low", "close", "volume"]]

#print the first 5 rows of the dataframe
print(df_stock.head())

print("\n\n-----------------------------------\n\n")

print("Spotify News Data")
#lets clean the dataframe to have only the columns we need
df_news = df_news[["date", "headline", "summary"]]

#print the first 5 rows of the dataframe
print(df_news.head())

Spotify Stock Data
         date    open      high      low   close     volume
0  2025-02-28  584.25  609.9200  580.000  608.01  4531895.0
1  2025-02-27  611.00  613.0000  586.000  590.76  1191837.0
2  2025-02-26  595.62  608.5294  592.890  603.13  2629946.0
3  2025-02-25  597.22  599.1200  575.535  588.57  2822820.0
4  2025-02-24  612.30  621.9100  592.980  601.61  2078629.0


-----------------------------------


Spotify News Data
         date                                           headline  \
0  2025-02-28  Palantir, Nvidia Nixed But Netflix Remains On ...   
1  2025-02-27  Tracking Chase Coleman's Tiger Global Portfoli...   
2  2025-02-27  Spotify Technology (SPOT) is Attracting Invest...   
3  2025-02-27  YouTube Surpasses Competitors in Streaming and...   
4  2025-02-26  Spotify CEO Wants EU To Penalize Apple For Def...   

                                             summary  
0  A volatile market has shaken off Nvidia and Pa...  
1  Tiger Global's 13F reveals a $26.46B port

In [64]:
# Convert dates to datetime format
df_stock['date'] = pd.to_datetime(df_stock['date'])
df_news['date'] = pd.to_datetime(df_news['date'])

# Create complete date range DataFrame
start_date = min(df_stock['date'].min(), df_news['date'].min())
end_date = max(df_stock['date'].max(), df_news['date'].max())
date_range = pd.DataFrame({'date': pd.date_range(start=start_date, end=end_date)})

# Merge stock data with complete date range and forward fill missing values
complete_stock = pd.merge(date_range, df_stock, on='date', how='left')
complete_stock = complete_stock.sort_values('date')
complete_stock = complete_stock.ffill()  # Forward fill to use previous day's data for missing dates

# Add indicator for trading days
complete_stock['is_trading_day'] = complete_stock['date'].isin(df_stock['date'])

# Now merge with news data, keeping one row per news item
df_merged = pd.merge(complete_stock, df_news, on='date', how='outer')

# Sort by date
df_merged = df_merged.sort_values('date')

# Fill NaN values in news columns
df_merged['headline'] = df_merged['headline'].fillna('')
df_merged['summary'] = df_merged['summary'].fillna('')

print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1444 entries, 0 to 1443
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1444 non-null   datetime64[ns]
 1   open            1444 non-null   float64       
 2   high            1444 non-null   float64       
 3   low             1444 non-null   float64       
 4   close           1444 non-null   float64       
 5   volume          1444 non-null   float64       
 6   is_trading_day  1444 non-null   bool          
 7   headline        1444 non-null   object        
 8   summary         1444 non-null   object        
dtypes: bool(1), datetime64[ns](1), float64(5), object(2)
memory usage: 102.9+ KB
None


In [65]:
# Count duplicated headlines and summaries
headline_counts = df_merged['headline'].value_counts()
duplicated_headlines = headline_counts[headline_counts > 1]
print(f"Total number of duplicated headlines: {len(duplicated_headlines)}")
print("\nTop 10 most duplicated headlines:")
print(duplicated_headlines.head(10))

print("\n" + "-"*50 + "\n")

summary_counts = df_merged['summary'].value_counts()
duplicated_summaries = summary_counts[summary_counts > 1]
print(f"Total number of duplicated summaries: {len(duplicated_summaries)}")
print("\nTop 10 most duplicated summaries:")
print(duplicated_summaries.head(10))

# Display the actual headlines/summaries with their dates
print("\n" + "-"*50 + "\n")
print("Details of most duplicated headline:")
if len(duplicated_headlines) > 0:
    most_dup_headline = duplicated_headlines.index[0]
    print(f"Headline (appearing {duplicated_headlines.iloc[0]} times): {most_dup_headline}")
    print("\nDates when this headline appeared:")
    for date in df_merged[df_merged['headline'] == most_dup_headline]['date']:
        print(f"- {date}")

print("\n" + "-"*50 + "\n")
print("Details of most duplicated summary:")
if len(duplicated_summaries) > 0:
    most_dup_summary = duplicated_summaries.index[0]
    print(f"Summary (appearing {duplicated_summaries.iloc[0]} times): {most_dup_summary}")
    print("\nDates when this summary appeared:")
    for date in df_merged[df_merged['summary'] == most_dup_summary]['date']:
        print(f"- {date}")


Total number of duplicated headlines: 32

Top 10 most duplicated headlines:
headline
                                                                                            73
Spotify Technology (SPOT) is Attracting Investor Attention: Here is What You Should Know     5
What You Missed On Wall Street This Morning                                                  4
Spotify Technology (SPOT) Is a Trending Stock: Facts to Know Before Betting on It            4
Is Trending Stock Spotify Technology (SPOT) a Buy Now?                                       3
What You Missed On Wall Street On Tuesday                                                    3
Morgan Stanley Reaffirms Their Buy Rating on Spotify Technology SA (SPOT)                    3
Spotify Technology S.A. (SPOT): A Bull Case Theory                                           3
A Closer Look at Spotify Technology's Options Market Dynamics                                3
Spotify price target raised by $50 at KeyBanc, here's why   

In [66]:
#lets delete the duplicates headlines and summaries from the dataframe
df_merged = df_merged.drop_duplicates(subset=['headline', 'summary'])

print(df_merged.head())
print(df_merged.info())

         date    open    high     low   close     volume  is_trading_day  \
0  2024-03-25  263.01  264.95  260.89  261.92   824685.0            True   
11 2024-03-27  267.00  269.72  257.56  260.20  1427218.0            True   
10 2024-03-27  267.00  269.72  257.56  260.20  1427218.0            True   
8  2024-03-27  267.00  269.72  257.56  260.20  1427218.0            True   
7  2024-03-27  267.00  269.72  257.56  260.20  1427218.0            True   

                                             headline  \
0                                                       
11  KeyBanc Keeps Their Buy Rating on Spotify Tech...   
10  Taylor Swift Had A Monster Year, As Did The Mu...   
8   Another List Throws Netflix (NASDAQ:NFLX) Out,...   
7   Spotify initiated with bullish view at HSBC, h...   

                                              summary  
0                                                      
11  Looking for stock market analysis and research...  
10  Looking for stock market ana

In [67]:
# First, identify the specific summaries and headlines to completely remove
summaries_to_remove = [
    "Looking for stock market analysis and research with proves results? Zacks.com offers in-depth financial research with over 30years of proven results.",
    ""  # Empty or whitespace summary
]

headlines_to_remove = [""]  # Empty or whitespace headline

# Get the original dataframe size
original_size = len(df_merged)
print(f"Original dataframe size: {original_size} rows")

# Step 1: Completely remove rows with the specified summaries
df_cleaned = df_merged[~df_merged['summary'].isin(summaries_to_remove)]
step1_size = len(df_cleaned)
print(f"After removing specific summaries: {step1_size} rows (removed {original_size - step1_size} rows)")

# Step 2: Completely remove rows with the specified headlines
df_cleaned = df_cleaned[~df_cleaned['headline'].isin(headlines_to_remove)]
step2_size = len(df_cleaned)
print(f"After removing specific headlines: {step2_size} rows (removed {step1_size - step2_size} rows)")

# Step 3: For other duplicated headlines, keep only the first occurrence
df_cleaned = df_cleaned.drop_duplicates(subset=['headline'], keep='first')
step3_size = len(df_cleaned)
print(f"After removing duplicate headlines: {step3_size} rows (removed {step2_size - step3_size} rows)")

# Step 4: For other duplicated summaries, keep only the first occurrence
df_cleaned = df_cleaned.drop_duplicates(subset=['summary'], keep='first')
final_size = len(df_cleaned)
print(f"After removing duplicate summaries: {final_size} rows (removed {step3_size - final_size} rows)")

print(f"\nTotal rows removed: {original_size - final_size}")
print(f"Final dataframe size: {final_size} rows")

# Check if there are any remaining duplicates
remaining_dup_headlines = df_cleaned['headline'].duplicated().sum()
remaining_dup_summaries = df_cleaned['summary'].duplicated().sum()
print(f"\nRemaining duplicated headlines: {remaining_dup_headlines}")
print(f"Remaining duplicated summaries: {remaining_dup_summaries}")

Original dataframe size: 1344 rows
After removing specific summaries: 910 rows (removed 434 rows)
After removing specific headlines: 910 rows (removed 0 rows)
After removing duplicate headlines: 896 rows (removed 14 rows)
After removing duplicate summaries: 879 rows (removed 17 rows)

Total rows removed: 465
Final dataframe size: 879 rows

Remaining duplicated headlines: 0
Remaining duplicated summaries: 0


In [68]:
#Lets check if there are still any duplicates in the dataframe

# Count duplicated headlines and summaries
headline_counts = df_cleaned['headline'].value_counts()
duplicated_headlines = headline_counts[headline_counts > 1]
print(f"Total number of duplicated headlines: {len(duplicated_headlines)}")
print("\nTop 10 most duplicated headlines:")
print(duplicated_headlines.head(10))

print("\n" + "-"*50 + "\n")

summary_counts = df_cleaned['summary'].value_counts()
duplicated_summaries = summary_counts[summary_counts > 1]
print(f"Total number of duplicated summaries: {len(duplicated_summaries)}")
print("\nTop 10 most duplicated summaries:")
print(duplicated_summaries.head(10))

# Display the actual headlines/summaries with their dates
print("\n" + "-"*50 + "\n")
print("Details of most duplicated headline:")
if len(duplicated_headlines) > 0:
    most_dup_headline = duplicated_headlines.index[0]
    print(f"Headline (appearing {duplicated_headlines.iloc[0]} times): {most_dup_headline}")
    print("\nDates when this headline appeared:")
    for date in df_cleaned[df_cleaned['headline'] == most_dup_headline]['date']:
        print(f"- {date}")

print("\n" + "-"*50 + "\n")
print("Details of most duplicated summary:")
if len(duplicated_summaries) > 0:
    most_dup_summary = duplicated_summaries.index[0]
    print(f"Summary (appearing {duplicated_summaries.iloc[0]} times): {most_dup_summary}")
    print("\nDates when this summary appeared:")
    for date in df_cleaned[df_cleaned['summary'] == most_dup_summary]['date']:
        print(f"- {date}")

Total number of duplicated headlines: 14

Top 10 most duplicated headlines:
headline
Spotify Technology S.A. (SPOT): A Bull Case Theory                                          3
Spotify Technology (SPOT) is Attracting Investor Attention: Here is What You Should Know    2
Spotify (SPOT) Declines More Than Market: Some Information for Investors                    2
Spotify Technology (SPOT) Is a Trending Stock: Facts to Know Before Betting on It           2
Spotify (SPOT) Surpasses Market Returns: Some Facts Worth Knowing                           2
Is Trending Stock Spotify Technology (SPOT) a Buy Now?                                      2
Wall Street Bulls Look Optimistic About Spotify (SPOT): Should You Buy?                     2
TMUS, GOOGL, SPOT: Which “Strong Buy” Communications Stock Is the Best Bet?                 2
Is Spotify Technology (SPOT) Stock Outpacing Its Business Services Peers This Year?         2
Spotify (SPOT) Sees a More Significant Dip Than Broader Market: Some 

In [69]:
# Save the merged dataframe
df_cleaned.to_csv("/Users/armandocriscuolo/c2025/data_science_project_2025/code/Data-Science-Project/Data-Question-2-B/news_analysis/spotify_news_stock_data_cleaned.csv", index=False)